In [2]:
!pip install librosa

    100% |████████████████████████████████| 1.6MB 21.2MB/s ta 0:00:01
    100% |████████████████████████████████| 286kB 37.9MB/s ta 0:00:01
    100% |████████████████████████████████| 327kB 38.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for librosa ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
  Running setup.py bdist_wheel for audioread ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/53/02/90/7b5c4081b7470c550ab605f600bad237dde12a6b8999b11f50
  Running setup.py bdist_wheel for resampy ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built librosa audioread resampy
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [42]:
import numpy as np
import librosa
import os
import glob
import pandas as pd

In [75]:
folds = ['/home/ubuntu/UrbanSound8K/audio/fold1', '/home/ubuntu/UrbanSound8K/audio/fold2', 
               '/home/ubuntu/UrbanSound8K/audio/fold3', '/home/ubuntu/UrbanSound8K/audio/fold4',
                '/home/ubuntu/UrbanSound8K/audio/fold5', '/home/ubuntu/UrbanSound8K/audio/fold6',
                '/home/ubuntu/UrbanSound8K/audio/fold7', '/home/ubuntu/UrbanSound8K/audio/fold8',
                '/home/ubuntu/UrbanSound8K/audio/fold9', '/home/ubuntu/UrbanSound8K/audio/fold10']

# for fold in folds:
#     files = os.listdir(fold)
    
#     for file in files:
#         print(file)
#         X, sample_rate = librosa.load(fold + '/' + file)
#         break
    
audio = '/home/ubuntu/UrbanSound8K/audio'
folds = ["fold1","fold2","fold3","fold4","fold5","fold6","fold7","fold8","fold9","fold10"]

def normalize(feature):
    mean = np.mean(feature, axis=0)
    std = np.std(feature, axis=0, ddof=1)
    feature -= mean
    feature /= std
    return feature



def get_audio_features(parent_dir, sub_dirs, file_ext="*.wav"):
    result = np.empty((0,59))
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            librosa_load, librosa_sampling_rate = librosa.load(fn, mono=True)
            stft = np.abs(librosa.stft(librosa_load))
            mfccs = np.mean(librosa.feature.mfcc(y=librosa_load, sr=librosa_sampling_rate, n_mfcc=40).T,axis=0)
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=librosa_sampling_rate).T,axis=0)
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=librosa_sampling_rate).T,axis=0)
            mfcc_tot = normalize(mfccs)
            chroma_tot = normalize(chroma)
            contrast_tot = normalize(contrast)
            feature_cols = np.hstack([mfcc_tot, chroma_tot, contrast_tot])
            #labels = np.append(labels, fn.split('/')[2].split('-')[1])
            result = np.vstack((result, feature_cols))
    return result
np_out = get_audio_features(audio, folds)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/home/ubuntu/anaconda3/lib/python3.7/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/home/ubuntu/anaconda3/lib/python3.7/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/home/ubuntu/anaconda3/lib/python3.7/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/home/ubuntu/anaconda3/lib/python3.7/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
 

In [76]:
len(np_out)

8732

In [78]:
# np.savetxt("features.csv", np_out, delimiter=",")

In [79]:
df = pd.read_csv('./features.csv')

In [82]:
df.head()

,-3.474220670334815964e+00,2.857897970375768981e+00,-3.590239939233062838e+00,1.746021854093317449e+00,-9.948786878218937035e-01,6.316859517701494120e-01,-6.917352256234181596e-01,3.332999351777252706e-01,-2.457553010090723633e-01,5.544724016360862845e-02,...,-6.867144938376570362e-01,-4.853452426389289753e-01,7.340359895996497741e-01,-2.705942364236723630e-01,-5.880346485479942542e-01,-5.869473092834435191e-01,-6.011493100330262962e-01,-4.840346262677158173e-01,4.173106589610258155e-01,2.113449471594827322e+00
0,-5.373538,2.919077,-0.661504,0.601540,0.045294,0.148810,0.424438,0.408690,0.343655,0.316244,...,1.535709,0.588437,-0.513087,-1.036728,-0.957482,-0.267436,0.025869,-0.010860,0.282955,1.963683
1,-2.552239,4.187415,-3.076004,0.028714,-0.860569,-0.170561,-0.745565,-0.078530,-0.903930,-0.077292,...,-1.310567,0.259663,1.451087,-0.747415,-0.585809,-0.460454,-0.469024,-0.167301,0.296919,2.133084
2,-6.058441,1.115634,0.220812,0.633115,0.193441,-0.011261,0.013524,-0.009777,0.005731,-0.005094,...,-1.345980,-0.263892,0.123302,-0.319602,-0.743017,-0.097980,-0.570727,-0.337132,-0.139980,2.208438
3,-3.557316,4.863821,-1.032547,0.378554,0.261618,0.140604,0.189797,0.177898,-0.032236,0.470381,...,1.507666,1.346528,0.336574,0.113144,-1.086707,-0.487760,-0.447089,-0.178485,0.004448,2.082449
4,-5.919869,1.461585,-0.569476,-0.101312,-0.131756,0.502519,0.554460,0.487312,0.079839,0.261224,...,-0.942560,-0.866023,-0.433400,-0.581125,-1.364023,-0.947368,0.517501,0.243292,0.675280,1.456443
